In [1]:
from hume import HumeVoiceClient, MicrophoneInterface
from dotenv import load_dotenv
import os
import websockets
import base64
import wave
import json
import asyncio
from pydub import AudioSegment
from io import BytesIO

load_dotenv()

from VoiceActivityDetector import VoiceActivityDetector
from openai import AsyncOpenAI

client = AsyncOpenAI()

import nest_asyncio 
nest_asyncio.apply()

Start speaking.


In [6]:
async def text_chunker(chunks):
    """Yield complete sentences from streamed text input."""
    sentence_endings = (".", "?", "!")
    buffer = ""

    async for text in chunks:
        message = text.choices[0].delta.content
        if message == None:
            continue
        buffer += message

        # Find the last occurrence of any sentence-ending punctuation in the buffer
        last_punct = max(buffer.rfind(punct) for punct in sentence_endings)

        if last_punct != -1:
            # If there is sentence-ending punctuation, split at the last punctuation
            sentence = buffer[: last_punct + 1]  # Include the punctuation in the output
            yield sentence
            buffer = buffer[
                last_punct + 1 :
            ].strip()  # Save what's after the punctuation into the buffer

    # If there's any remaining text in the buffer after the loop, yield it
    if buffer:
        yield buffer

In [7]:
async def websocket_send(websocket, model_generator):
    # Iterate over the asynchronous generator from the OpenAI API
    async for message in text_chunker(model_generator):
        print("Sent message:", message)
        # Here, we're assuming message has a structure we can directly send
        await websocket.send(
            json.dumps(
                {"type": "assistant_input", "text": message}
            )
        )

In [8]:
async def receive_and_save_audio(websocket, filename):
    # Set up the WAV file for writing audio output
    audiosegment = AudioSegment.empty()
    try:
        async for message in websocket:
            print("Received message")
            data = json.loads(message)
            if data["type"] == "audio_output":
                # Convert base64 audio data to bytes
                data = base64.b64decode(data["data"])
                audiosegment += AudioSegment.from_file(BytesIO(data))
            else:
                print("Received non-audio message")
    finally:
        audiosegment.export(filename, format="wav")

In [9]:
HUME_API_KEY = os.getenv("HUME_API_KEY")
websocket_url = f"wss://api.hume.ai/v0/evi/chat?api_key={HUME_API_KEY}"
wave_file = "test.wav"
async with websockets.connect(websocket_url) as websocket:
    response = await client.chat.completions.create(
        model="gpt-4-turbo",
        stream=True,
        max_tokens=500,
        messages=[
            {"role": "system", "content": "You are a storyteller."},
            {
                "role": "user",
                "content": "Tell me a emotional short story about a ship for a four year old.",
            },
        ],
    )

    sender_task = asyncio.create_task(websocket_send(websocket, response))
    receiver_task = asyncio.create_task(receive_and_save_audio(websocket, wave_file))
    await asyncio.gather(sender_task, receiver_task)

Sent message: Once upon a time, there was a small, blue wooden ship named Sally.
Sent message:  Sally wasn't like the big ships that carried cars or the tall ships that had many sails.
Received message
Received non-audio message
Received message
Sent message:  She was a simple ship, perfect for bobbing along the gentle rivers and exploring the hidden secrets of the water.
Received message
Received non-audio message
Received message
Received message
Received non-audio message
Received message
Sent message: Sally had one very special friend, Timmy, a cheerful little boy with a wide smile and sparkly eyes.
Received message
Received non-audio message
Received message
Sent message:  Every summer, Timmy would come to the river with his grandpa, and the first thing he would do was run to Sally, greeting her with a big, warm hug.
Sent message: "Hello Sally!
Sent message:  Are you ready for an adventure?
Sent message: " Timmy would shout, and it seemed like Sally shook her mast in excitement.
R

CancelledError: 